In [31]:
#importing required libraries and packages
import glob
import numpy as np
import os
import mxnet as mx
from mxnet import gluon
from PIL import Image
from scipy import signal
from matplotlib import pyplot as plt
print("libraries imported successfully !!")

libraries imported successfully !!


In [32]:
#Importing our dataset from ucsd anaomaly site 
tar = 'C:\\Users\\Shrishti D Hore\\OneDrive\\Documents\\HEU_AI\\UCSD_Anomaly_Dataset.v1p2'
print("dataset imported successfully !!")

dataset imported successfully !!


In [39]:
#We are using Convolutional Autoencoder(CAE) for good reason that they can be trained in normal parts and wont require annotated data.
#Once trained we can give a featured representation for a part and compare autoencoder output and input
#Concept is the larger the difference the more likely the anomaly
#Two parts of the autoencoder : encoder and decoder
#Here encoder will encode the input data using a reduced representation and the decoder will attempt to re-construct the original input data from reduced representation
#In our CAE model encoder will consist of 2 convolutional and 2 Max-Pooling layers and decoder will consists of two Unsampling and Two Deconvolutions
class ConvolutionalAutoencoder(gluon.nn.HybridBlock):
    
    def __init__(self):
        super(ConvolutionalAutoencoder, self).__init__()
        
        with self.name_scope():
            self.encoder = gluon.nn.HybridSequential() #for stacking the hybrid blocks sequentially
            with self.encoder.name_scope():
                self.encoder.add(gluon.nn.Conv2D(32, 5, activation='relu')) #2D convolution layer with 32 channels and kernel_size set to 5 with activation function as Relu
                self.encoder.add(gluon.nn.MaxPool2D(2)) #maxpooling for our 2D spatial data
                self.encoder.add(gluon.nn.Conv2D(32, 5, activation='relu')) 
                self.encoder.add(gluon.nn.MaxPool2D(2))
                self.encoder.add(gluon.nn.Dense(2000))

            self.decoder = gluon.nn.HybridSequential()
            with self.decoder.name_scope():
                self.decoder.add(gluon.nn.Dense(32*22*22, activation='relu'))
                self.decoder.add(gluon.nn.HybridLambda(lambda F, x: F.UpSampling(x, scale=2, sample_type='nearest'))) #taking a lambda function to wrap our operations as a Hybridblock object
                self.decoder.add(gluon.nn.Conv2DTranspose(32, 5, activation='relu'))
                self.decoder.add(gluon.nn.HybridLambda(lambda F, x: F.UpSampling(x, scale=2, sample_type='nearest')))
                self.decoder.add(gluon.nn.Conv2DTranspose(1, kernel_size=5, activation='sigmoid')) #1 dense layer with kernel size to 5 and activation function as sigmoid

    def hybrid_forward(self, F, x):
        x = self.encoder(x)
        x = self.decoder[0](x)
        
        #needs to be reshaped output vector from Dense(32*22*22), before it is unsampled
        x = x.reshape((-1,32,22,22))
        x = self.decoder[1:](x)

        return x
print("Encoder and decoder layers set for the neural network!!!!")

Encoder and decoder layers set for the neural network!!!!


In [40]:
#Training the Autoencoder for 30 epochs and setting the batch size to 32
#Main traing loop for the CAE using previously set epochs and batch size
#Here we will compute loss function backwards computing dloss/dx for every parameter 
#Here the optimizer step will update the value of parameter using gradient 

ctx = mx.cpu()
num_epochs = 30
batch_size = 32

# Train the autoencoder
def train(batch_size, ctx, num_epochs, path, lr=1e-4, wd=1e-5, params_file="autoencoder_ucsd_convae.params"):
    
    # Dataloader for training dataset
    dataloader = utils.create_dataset(path, batch_size, shuffle=True)
    
    # Get model
    model = ConvolutionalAutoencoder()
    model.hybridize()
    
    # Initialiize
    model.collect_params().initialize(mx.init.Xavier('gaussian'), ctx=ctx)
    
    # Loss
    l2loss = gluon.loss.L2Loss()
    optimizer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd})
    
    # Start training loop
    for epoch in range(num_epochs):

        
        for image in dataloader:
            
            image = image.as_in_context(ctx)

            with mx.autograd.record():
                
                reconstructed = model(image)
                loss = l2loss(reconstructed, image)

            loss.backward()
            optimizer.step(batch_size)
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, mx.nd.mean(loss).asscalar()))
    
    # Save parameters
    model.save_parameters(params_file)
    return model, params_file

print("training of autoencoder with bachsize 32 set !!!")

training of autoencoder with bachsize 32 set !!!


In [41]:
#SInce the images in the UCSDpeds1 folder in UCSD_Anomaly_Dataset.v1p2 main folder have the image format of 158x238 pixels
#We need to rescale the images to 100x100 and normalize them
files = sorted(glob.glob('UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/*/*'))

a = np.zeros((len(files),1,100,100))

for idx, filename in enumerate(files):
    im = Image.open('C:\\Users\\Shrishti D Hore\\OneDrive\\Documents\\HEU_AI\\UCSD_Anomaly_Dataset.v1p2\\UCSDped1\\Train\\Train001')
    im = im.resize((100,100))
    a[idx,0,:,:] = np.array(im, dtype=np.float32)/255.0

dataset = gluon.data.ArrayDataset(mx.nd.array(a, dtype=np.float32))
dataloader = gluon.data.DataLoader(dataset, batch_size=batch_size, last_batch='rollover',shuffle=True)
print("Rescaled and normalized images successfully !!!")

Rescaled and normalized images successfully !!!


In [42]:
#Now we need to initialize and define our loss function of the Convolutional Autoencoder
model = ConvolutionalAutoencoder()
model.hybridize()
model.collect_params().initialize(mx.init.Xavier('gaussian'), ctx=ctx)
loss_function = gluon.loss.L2Loss()
optimizer = gluon.Trainer(model.collect_params(), 'adam', {'learning_rate': 1e-4, 'wd': 1e-5})
print("defined loss function successfully !!!")

defined loss function successfully !!!
